In [1]:
# importing libraries
import requests
from bs4 import BeautifulSoup
from newspaper import Article  
import csv 
import pandas as pd
import numpy as np
import nltk
# nltk.download('all') #(Download the package if not downloaded already - Large file) #uncomment it out

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [2]:
#crawling using beautiful soup

url="https://in.mashable.com/"
r=requests.get(url)
soup=BeautifulSoup(r.content,'html5lib') 
table=soup.findAll('a',attrs={'class':'title'})
news=[]
for row in table: 
    if not row['href'].startswith('http'):
        news.append('https://in.mashable.com'+row['href'])

df=[]
for i in news:
    article = Article(i, language="en")
    article.download() 
    article.parse() 
    article.nlp() 
    data={}
    data['Title']=article.title
    data['Text']=article.text
    data['Summary']=article.summary
    data['Keywords']=article.keywords
    df.append(data)

    
dataset=pd.DataFrame(df)
dataset.head(10)

,Title,Text,Summary,Keywords
0,'Anwar Ka Ajab Kissa' Review: Nawazuddin Siddi...,"Anwar Ka Ajab Kissa or Sniffer, written and di...","Anwar Ka Ajab Kissa or Sniffer, written and di...","[movie, ka, anwar, review, dont, siddiqui, kno..."
1,COVID-19: Oxford Vaccine To Be Available In In...,And after months of waiting while Coronavirus ...,Pardon my philosophy but the pandemic isn’t ov...,"[1000, 2021, indian, today, poonawalla, result..."
2,Facebook Messenger Patched A Bug That Would Al...,One of the most widely-known spying methods is...,"Fret not, the bug has been patched though, cla...","[snooping, facebook, attacker, messenger, rese..."
3,Amazon's Making An Upgraded Version Of Its Ech...,Amazon announced today that its experimental s...,"Echo Frames have now exited its testing phase,...","[upgraded, smart, echo, devices, volume, assis..."
4,NVIDIA GeForce NOW Streaming Service Comes To ...,NVIDIA is using a web-based client to bypass A...,NVIDIA is using a web-based client to bypass A...,"[brings, game, google, cloud, geforce, streami..."
5,Microsoft Teams Now Offers 24-Hour Video Calls...,"Just in time for the holiday season, Microsoft...","Just in time for the holiday season, Microsoft...","[microsoft, zoom, offers, start, calls, 24hour..."
6,Netflix For Free In India? All You Need To Kno...,Nothing grabs eyeballs like the word FREE does...,Netflix knows this and it's planning to do som...,"[streamfest, need, sign, netflix, streaming, k..."
7,‘Godmothered’ Trailer: Jillian Bell Gives Isla...,‘Tis the season to be jolly and it’s raining h...,"In fact, if you’ve watched and liked Enchanted...","[gives, bell, fairy, need, tale, eleanor, jone..."
8,'Middle Class Melodies' Review: A Mediocre Mov...,"Middle Class Melodies, directed by Vinod Anant...","Now, Middle Class Melodies tries to encapsulat...","[movie, middle, melodies, mediocrity, writing,..."
9,Akshay Kumar Serves Rs 500 Crore Defamation No...,The Sushant Singh Rajput death case opened a c...,The Sushant Singh Rajput death case opened a c...,"[case, youtuber, singh, client, false, kumar, ..."


In [3]:
FILEPATH="OnlineNewsPopularity.csv" #dataset from UCI Repo
data1=pd.read_csv(FILEPATH)
data1.head(10)

,url,timedelta,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_self_hrefs,num_imgs,...,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity,shares
0,http://mashable.com/2013/01/07/amazon-instant-...,731.0,12.0,219.0,0.663594,1.0,0.815385,4.0,2.0,1.0,...,0.100000,0.7,-0.350000,-0.600,-0.200000,0.500000,-0.187500,0.000000,0.187500,593
1,http://mashable.com/2013/01/07/ap-samsung-spon...,731.0,9.0,255.0,0.604743,1.0,0.791946,3.0,1.0,1.0,...,0.033333,0.7,-0.118750,-0.125,-0.100000,0.000000,0.000000,0.500000,0.000000,711
2,http://mashable.com/2013/01/07/apple-40-billio...,731.0,9.0,211.0,0.575130,1.0,0.663866,3.0,1.0,1.0,...,0.100000,1.0,-0.466667,-0.800,-0.133333,0.000000,0.000000,0.500000,0.000000,1500
3,http://mashable.com/2013/01/07/astronaut-notre...,731.0,9.0,531.0,0.503788,1.0,0.665635,9.0,0.0,1.0,...,0.136364,0.8,-0.369697,-0.600,-0.166667,0.000000,0.000000,0.500000,0.000000,1200
4,http://mashable.com/2013/01/07/att-u-verse-apps/,731.0,13.0,1072.0,0.415646,1.0,0.540890,19.0,19.0,20.0,...,0.033333,1.0,-0.220192,-0.500,-0.050000,0.454545,0.136364,0.045455,0.136364,505
5,http://mashable.com/2013/01/07/beewi-smart-toys/,731.0,10.0,370.0,0.559889,1.0,0.698198,2.0,2.0,0.0,...,0.136364,0.6,-0.195000,-0.400,-0.100000,0.642857,0.214286,0.142857,0.214286,855
6,http://mashable.com/2013/01/07/bodymedia-armba...,731.0,8.0,960.0,0.418163,1.0,0.549834,21.0,20.0,20.0,...,0.100000,1.0,-0.224479,-0.500,-0.050000,0.000000,0.000000,0.500000,0.000000,556
7,http://mashable.com/2013/01/07/canon-poweshot-n/,731.0,12.0,989.0,0.433574,1.0,0.572108,20.0,20.0,20.0,...,0.100000,1.0,-0.242778,-0.500,-0.050000,1.000000,0.500000,0.500000,0.500000,891
8,http://mashable.com/2013/01/07/car-of-the-futu...,731.0,11.0,97.0,0.670103,1.0,0.836735,2.0,0.0,0.0,...,0.400000,0.8,-0.125000,-0.125,-0.125000,0.125000,0.000000,0.375000,0.000000,3600
9,http://mashable.com/2013/01/07/chuck-hagel-web...,731.0,10.0,231.0,0.636364,1.0,0.797101,4.0,1.0,1.0,...,0.100000,0.5,-0.238095,-0.500,-0.100000,0.000000,0.000000,0.500000,0.000000,710


In [4]:
clean_cols={x: x.lower().strip() for x in list(data1)}
data1=data1.rename(index=str,columns=clean_cols)

In [5]:
train_set, test_set = train_test_split(data1, test_size=0.4, random_state=42)

drop_attributes_list = ['url','shares', 'timedelta','num_self_hrefs', 'kw_min_min', 'kw_max_min', 'kw_avg_min','kw_min_max','kw_max_max','kw_avg_max','kw_min_avg','kw_max_avg','kw_avg_avg','self_reference_min_shares','self_reference_max_shares','self_reference_avg_sharess']

x_train = train_set.drop(drop_attributes_list, axis=1)
y_train = train_set['shares']

x_test = test_set.drop(drop_attributes_list, axis=1)
y_test = test_set['shares']



In [6]:
print("Training classifier...")
clf=RandomForestRegressor(n_estimators=10, random_state=42)
clf.fit(x_train,y_train)
print("Trained successful")

Training classifier...
Trained successful


In [7]:

result=pd.DataFrame(clf.predict(x_train),list(y_train))
result.reset_index(level=0, inplace=True)
result_df = result.rename(index=str, columns={"index": "Actual shares", 0: "Predicted shares"})
result_df.head(10)

,Actual shares,Predicted shares
0,1900,2200.0
1,1800,1680.0
2,790,841.0
3,872,884.0
4,2500,2383.6
5,860,1750.0
6,1000,1047.8
7,47300,25569.6
8,20400,16584.5
9,2500,2470.0


In [8]:
FILEPATH2="abcnews-date-text.csv"
data = pd.read_csv(FILEPATH2, error_bad_lines=False);
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text
print("Number of headlines in the dataset:", len(documents))
# print(len(documents))
# print(documents[:5])
documents.head(10)

Number of headlines in the dataset: 1186018


,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4
5,ambitious olsson wins triple jump,5
6,antic delighted with record breaking barca,6
7,aussie qualifier stosur wastes four memphis match,7
8,aust addresses un security council over iraq,8
9,australia is locked into war timetable opp,9


In [9]:
#importing libraries
#pip install gensim (install gensim packages if you havent installed it already)
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)
import nltk
stemmer=PorterStemmer()

In [10]:
#Lemmatizing the text of the article - breaking into base words follwed by stemming 
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
#Removing the stopword and considering only token length > 3 
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [11]:
#example of how stemming and lemmatizing is being done by taking an example from the dataset. 
doc_sample = documents[documents['index'] ==2].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n')
print('tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['a', 'g', 'calls', 'for', 'infrastructure', 'protection', 'summit']


tokenized and lemmatized document: 
['call', 'infrastructur', 'protect', 'summit']


In [12]:
processed_docs = documents['headline_text'].map(preprocess)
processed_docs[:10]




0               [decid, commun, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [13]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print(dictionary)
count=0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 15:
        break

Dictionary(67259 unique tokens: ['broadcast', 'commun', 'decid', 'licenc', 'awar']...)
0 broadcast
1 commun
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit
11 aust
12 rise
13 staff
14 strike
15 affect


In [18]:
# doc2bow is bag of words. Assgin the count or freq of the tokens(given by unique id in dictionary)
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[2]

[(7, 1), (8, 1), (9, 1), (10, 1)]

In [27]:
#printing the example of doc2bow
bow_doc_2 = bow_corpus[2]
for i in range(len(bow_doc_2)):
    print("Word {} i.e. (\"{}\") appears {} time.".format(bow_doc_2[i][0], 
                                               dictionary[bow_doc_2[i][0]], 
bow_doc_2[i][1]))

Word 7 i.e. ("call") appears 1 time.
Word 8 i.e. ("infrastructur") appears 1 time.
Word 9 i.e. ("protect") appears 1 time.
Word 10 i.e. ("summit") appears 1 time.


In [30]:
#map the entire corpus into num_topics = 5 topics
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=5, id2word=dictionary)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    
#It has picked up most the most contributing words/tokens with probabilty 
#Probabilty (which represent the contribution of that word towards the topic or corpus)


Topic: 0 
Words: 0.010*"plan" + 0.009*"govern" + 0.008*"rural" + 0.008*"council" + 0.008*"interview" + 0.008*"farmer" + 0.008*"water" + 0.007*"countri" + 0.007*"fund" + 0.007*"queensland"
Topic: 1 
Words: 0.028*"polic" + 0.013*"death" + 0.011*"woman" + 0.010*"crash" + 0.009*"news" + 0.009*"south" + 0.009*"donald" + 0.009*"kill" + 0.009*"coast" + 0.009*"attack"
Topic: 2 
Words: 0.019*"australia" + 0.016*"trump" + 0.015*"year" + 0.010*"market" + 0.010*"world" + 0.008*"die" + 0.008*"live" + 0.007*"final" + 0.006*"test" + 0.006*"say"
Topic: 3 
Words: 0.024*"australian" + 0.009*"shoot" + 0.008*"say" + 0.008*"australia" + 0.008*"tasmania" + 0.007*"time" + 0.007*"royal" + 0.007*"elect" + 0.005*"drum" + 0.005*"open"
Topic: 4 
Words: 0.017*"charg" + 0.015*"court" + 0.013*"murder" + 0.010*"face" + 0.009*"health" + 0.009*"accus" + 0.007*"trial" + 0.007*"alleg" + 0.007*"child" + 0.006*"commun"


In [31]:
for index, score in sorted(lda_model[bow_corpus[2]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 5)))


Score: 0.8381300568580627	 
Topic: 0.010*"plan" + 0.009*"govern" + 0.008*"rural" + 0.008*"council" + 0.008*"interview"

Score: 0.040902089327573776	 
Topic: 0.024*"australian" + 0.009*"shoot" + 0.008*"say" + 0.008*"australia" + 0.008*"tasmania"

Score: 0.040536362677812576	 
Topic: 0.019*"australia" + 0.016*"trump" + 0.015*"year" + 0.010*"market" + 0.010*"world"

Score: 0.04042215272784233	 
Topic: 0.017*"charg" + 0.015*"court" + 0.013*"murder" + 0.010*"face" + 0.009*"health"

Score: 0.040009383112192154	 
Topic: 0.028*"polic" + 0.013*"death" + 0.011*"woman" + 0.010*"crash" + 0.009*"news"


In [32]:

def get_LDA(text):
    unseen_document = text
    bow_vector = dictionary.doc2bow(preprocess(unseen_document))
    result=[]
    for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
        #print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))
        result.append(score)
    return result

In [42]:
from nltk.tokenize import word_tokenize #extract the tokens(words) from string of characters
from nltk.corpus import stopwords #Stopwords are the English words which does not add much meaning to a sentence.
stopwords=set(stopwords.words('english'))

from textblob import TextBlob #library for processing textual data. It provides a simple API for diving into common natural language processing (NLP) tasks such as part-of-speech tagging, noun phrase extraction, sentiment analysis, classification, translation, and more
import datefinder
import datetime  
from datetime import date

In [34]:
# Attributes functions
# n_unique_tokens: Rate of unique words in the content
def for_unique_tokens_rate(words):
    words=tokenize(words)
    no_order = list(set(words))
    unique_tokens_rate=len(no_order)/len(words)
    return unique_tokens_rate

# n_non_stop_words: Rate of non-stop words in the content
def for_nonstop_words_rate(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    nonstop_words_rate=len(filtered_sentence)/len(words)
    return nonstop_words_rate

# n_non_stop_unique_tokens: Rate of unique non-stop words in the content
def for_unique_nonstop_words_rate(words):
    words=tokenize(words)
    filtered_sentence = [w for w in words if not w in stopwords]
    no_order = list(set(filtered_sentence))
    unique_nonstop_words_rate=len(no_order)/len(words)
    return unique_nonstop_words_rate

# average_token_length: Average length of the words in the content
def avg_token_length(words):
    words=tokenize(words)
    length=[]
    for i in words:
        length.append(len(i))
    return np.average(length)

def day(article_text):
    article=article_text
    if len(list(datefinder.find_dates(article_text)))>0:
        date=str(list(datefinder.find_dates(article_text))[0])
        date=date.split()
        date=date[0]
        year, month, day = date.split('-')     
        day_name = datetime.date(int(year), int(month), int(day)) 
        return day_name.strftime("%A")
    return "Monday" #else return default day as monday

def tokenize(text):
    text=text
    return word_tokenize(text)

# Non-neutral tokens are those tokens which are more likely to exhibit the following POS Tags: Nouns, Adjectives, Adverbs, Interjections
def non_neutral_tokens(text):
    processed_docs = preprocess(text)
    speech=nltk.pos_tag(processed_docs)
    tags=["NN","NNP","NNS","JJ","JJR","JJS","RB","RBR","RBS","UH"]
    speech_list=[i[0] for i in speech if i[1] in tags]
    return len(speech_list)

In [35]:
positive_words=[]
negative_words=[]
def polar(words):
    all_tokens=tokenize(words)
    for i in all_tokens:
        analysis=TextBlob(i)
        polarity=analysis.sentiment.polarity
        if polarity>0:
            positive_words.append(i)
        if polarity<0:
            negative_words.append(i)
    return positive_words,negative_words

In [37]:
# global_rate_positive_words: Rate of positive words in the content
# global_rate_negative_words: Rate of negative words in the content
# rate_positive_words: Rate of positive words among non-neutral tokens
# rate_negative_words: Rate of negative words among non-neutral tokens

def rates(words):
    words=polar(words)
    positive=words[0]
    negative=words[1]
    
    global_rate_positive_words=(len(positive)/len(words))/100
    global_rate_negative_words=(len(negative)/len(words))/100
    
    polar_positive=[]
    polar_negative=[]
    
    for i in positive:
        analysis=TextBlob(i)
        polar_positive.append(analysis.sentiment.polarity)
        avg_positive_polarity=analysis.sentiment.polarity # avg_positive_polarity: Avg. polarity of positive words
    for j in negative:
        analysis2=TextBlob(j)
        polar_negative.append(analysis2.sentiment.polarity)
        avg_negative_polarity=analysis2.sentiment.polarity # avg_negative_polarity: Avg. polarity of negative words
        
    min_positive_polarity=min(polar_positive)
    max_positive_polarity=max(polar_positive)
    min_negative_polarity=min(polar_negative)
    max_negative_polarity=max(polar_negative)
    avg_positive_polarity=np.average(polar_positive)
    avg_negative_polarity=np.average(polar_negative)
    return global_rate_positive_words,global_rate_negative_words,avg_positive_polarity,min_positive_polarity,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity

In [43]:
df1=[]
for i in news:
    pred_info={}
    article = Article(i, language="en") # en for English 
    article.download() 
    article.parse()
    analysis=TextBlob(article.text)
    polarity=analysis.sentiment.polarity
    title_analysis=TextBlob(article.title)
    pred_info['text']=article.text
    pred_info['n_tokens_title']=len(tokenize(article.title))
    pred_info['n_tokens_content']=len(tokenize(article.text))
    pred_info['n_unique_tokens']=for_unique_tokens_rate(article.text)
    pred_info['n_non_stop_words']=for_nonstop_words_rate(article.text)
    pred_info['n_non_stop_unique_tokens']=for_unique_nonstop_words_rate(article.text)
    pred_info['num_hrefs']=article.html.count("https://in.mashable.com")
    pred_info['num_imgs']=len(article.images)
    pred_info['num_videos']=len(article.movies)
    pred_info['average_token_length']=avg_token_length(article.text)
    pred_info['num_keywords']=len(article.keywords)
    
    
    # categorizing  of news article according to lifestyle, entertainment, business etc
    if "life-style" in article.url: 
        pred_info['data_channel_is_lifestyle']=1
    else:
        pred_info['data_channel_is_lifestyle']=0
    if "etimes" in article.url:
        pred_info['data_channel_is_entertainment']=1
    else:
        pred_info['data_channel_is_entertainment']=0
    if "business" in article.url:
        pred_info['data_channel_is_bus']=1
    else:
        pred_info['data_channel_is_bus']=0
    if "social media" or "facebook" or "whatsapp" in article.text.lower():
        data_channel_is_socmed=1
        data_channel_is_tech=0
        data_channel_is_world=0
    else:
        data_channel_is_socmed=0
    if ("technology" or "tech" in article.text.lower()) or ("technology" or "tech" in article.url):
        data_channel_is_tech=1
        data_channel_is_socmed=0
        data_channel_is_world=0
    else:
        data_channel_is_tech=0
    if "world" in article.url:
        data_channel_is_world=1
        data_channel_is_tech=0
        data_channel_is_socmed=0
    else:
        data_channel_is_world=0
        
    pred_info['data_channel_is_socmed']=data_channel_is_socmed
    pred_info['data_channel_is_tech']=data_channel_is_tech
    pred_info['data_channel_is_world']=data_channel_is_world
    
    if day(i)=="Monday":
        pred_info['weekday_is_monday']=1
    else:
        pred_info['weekday_is_monday']=0
    if day(i)=="Tuesday":
        pred_info['weekday_is_tuesday']=1
    else:
        pred_info['weekday_is_tuesday']=0
    if day(i)=="Wednesday":
        pred_info['weekday_is_wednesday']=1
    else:
        pred_info['weekday_is_wednesday']=0
    if day(i)=="Thursday":
        pred_info['weekday_is_thursday']=1
    else:
        pred_info['weekday_is_thursday']=0
    if day(i)=="Friday":
        pred_info['weekday_is_friday']=1
    else:
        pred_info['weekday_is_friday']=0
    if day(i)=="Saturday":
        pred_info['weekday_is_saturday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_saturday']=0
    if day(i)=="Sunday":
        pred_info['weekday_is_sunday']=1
        pred_info['is_weekend']=1
    else:
        pred_info['weekday_is_sunday']=0
        pred_info['is_weekend']=0
    
    #   LDA_00: Closeness to LDA topic 0
    #   LDA_01: Closeness to LDA topic 1
    #   LDA_02: Closeness to LDA topic 2
    #   LDA_03: Closeness to LDA topic 3
    #   LDA_04: Closeness to LDA topic 4
    #   we have consider 5 topics    
    pred_info['LDA_00']=get_LDA(article.text)[0]
    pred_info['LDA_01']=get_LDA(article.text)[1]
    pred_info['LDA_02']=get_LDA(article.text)[2]
    pred_info['LDA_03']=get_LDA(article.text)[3]
    pred_info['LDA_04']=get_LDA(article.text)[4]
    
    
    pred_info['global_subjectivity']=analysis.sentiment.subjectivity
    pred_info['global_sentiment_polarity']=analysis.sentiment.polarity
    pred_info['global_rate_positive_words']=rates(article.text)[0]
    pred_info['global_rate_negative_words']=rates(article.text)[1]
    pred_info['avg_positive_polarity']=rates(article.text)[2]
    pred_info['min_positive_polarity']=rates(article.text)[3]
    pred_info['max_positive_polarity']=rates(article.text)[4]
    pred_info['avg_negative_polarity']=rates(article.text)[5]
    pred_info['min_negative_polarity']=rates(article.text)[6]
    pred_info['max_negative_polarity']=rates(article.text)[7]
    pred_info['rate_positive_words']=len(polar(article.text)[0])/non_neutral_tokens(article.text)
    pred_info['rate_negative_words']=len(polar(article.text)[1])/non_neutral_tokens(article.text)
    pred_info['title_subjectivity']=title_analysis.sentiment.subjectivity
    pred_info['title_sentiment_polarity']=title_analysis.sentiment.polarity
    pred_info['abs_title_subjectivity']=abs(title_analysis.sentiment.subjectivity-0.5)
    pred_info['abs_title_sentiment_polarity']=abs(title_analysis.sentiment.polarity)
    df1.append(pred_info)

In [47]:
pred_df=pd.DataFrame(df1)
pred_test=pred_df.drop(['text'],axis=1)
pred_df.head()
# pred_test.head()

,n_tokens_title,n_tokens_content,n_unique_tokens,n_non_stop_words,n_non_stop_unique_tokens,num_hrefs,num_imgs,num_videos,average_token_length,num_keywords,...,max_positive_polarity,avg_negative_polarity,min_negative_polarity,max_negative_polarity,rate_positive_words,rate_negative_words,title_subjectivity,title_sentiment_polarity,abs_title_subjectivity,abs_title_sentiment_polarity
0,19,1723,0.329658,0.575160,0.272200,33,13,0,3.688915,0,...,1.0,-0.322988,-0.7,-0.05,0.746637,0.627803,0.466667,-0.3,0.033333,0.3
1,17,541,0.545287,0.619224,0.430684,33,9,0,4.349353,0,...,1.0,-0.319634,-0.7,-0.05,2.988372,1.918605,0.400000,0.4,0.100000,0.4
2,14,355,0.529577,0.633803,0.422535,33,8,0,4.490141,0,...,1.0,-0.300009,-0.7,-0.05,5.990476,3.619048,0.000000,0.0,0.500000,0.0
3,13,530,0.556604,0.667925,0.462264,33,10,0,4.422642,0,...,1.0,-0.288286,-0.7,-0.05,4.162437,2.335025,0.400000,0.4,0.100000,0.4
4,13,412,0.558252,0.691748,0.432039,33,10,0,4.131068,0,...,1.0,-0.303337,-1.0,-0.05,6.101351,3.918919,0.000000,0.0,0.500000,0.0


In [52]:
test2=pd.DataFrame(clf.predict(pred_test),pred_df['text'])
test2.reset_index(level=0, inplace=True)
test2 = test2.rename(index=str, columns={"index": "News", 0: "Virality"})
test2


,text,Virality
0,"Anwar Ka Ajab Kissa or Sniffer, written and di...",22643.7
1,And after months of waiting while Coronavirus ...,14517.2
2,One of the most widely-known spying methods is...,22943.0
3,Amazon announced today that its experimental s...,13048.8
4,NVIDIA is using a web-based client to bypass A...,12938.9
5,"Just in time for the holiday season, Microsoft...",16218.8
6,Nothing grabs eyeballs like the word FREE does...,17309.4
7,‘Tis the season to be jolly and it’s raining h...,7182.5
8,"Middle Class Melodies, directed by Vinod Anant...",17613.7
9,The Sushant Singh Rajput death case opened a c...,14724.6
